<a href="https://colab.research.google.com/github/Muthulakshmi-selvaraj/Muthulakshmi/blob/main/chatbots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas
!pip install numpy
!pip install scikit-learn
!pip install nltk
!pip install transformers
!pip install datasets
!pip install matplotlib
!pip install seaborn
!pip install rasa

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to C:\Users\NEXT
[nltk_data]     GEN\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to C:\Users\NEXT
[nltk_data]     GEN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# DATA EXPLORATION AND PREPROCESSING

In [ ]:
# Load dataset
file_path = 'project data in csv file format - Copy.csv'
data = pd.read_csv(file_path)


In [ ]:
# View the first few rows
print("Dataset Sample:")
print(data.head())

Dataset Sample:
  flags                                          utterance category  \
0    BM            I have problems with canceling an order    ORDER   
1   BIM  how can I find information about canceling ord...    ORDER   
2     B          I need help with canceling the last order    ORDER   
3   BIP  could you help me cancelling the last order I ...    ORDER   
4     B            problem with cancelling an order I made    ORDER   

         intent  
0  cancel_order  
1  cancel_order  
2  cancel_order  
3  cancel_order  
4  cancel_order  


In [ ]:
# Replace NaN values with an empty string and convert all to strings
data['utterance'] = data['utterance'].fillna('').astype(str)


In [ ]:
import re
from nltk.corpus import stopwords

def clean_text(text):
    if not isinstance(text, str):
        text = str(text)  # Convert to string if not already
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    text = text.strip()  # Remove leading and trailing spaces
    text = ' '.join(word for word in text.split() if word not in stopwords.words('english'))  # Remove stopwords
    return text


In [ ]:
data['cleaned_utterance'] = data['utterance'].apply(clean_text)


In [ ]:
print(data[['utterance', 'cleaned_utterance']].head())


                                           utterance  \
0            I have problems with canceling an order   
1  how can I find information about canceling ord...   
2          I need help with canceling the last order   
3  could you help me cancelling the last order I ...   
4            problem with cancelling an order I made   

                       cleaned_utterance  
0               problems canceling order  
1      find information canceling orders  
2         need help canceling last order  
3  could help cancelling last order made  
4          problem cancelling order made  


In [ ]:
# Clean text data
def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    text = text.strip()  # Remove leading and trailing spaces
    text = ' '.join(word for word in text.split() if word not in stopwords.words('english'))  # Remove stopwords
    return text

data['cleaned_utterance'] = data['utterance'].apply(clean_text)

In [ ]:
# Tokenize and convert to TF-IDF representation
vectorizer = TfidfVectorizer(max_features=1000)  # Limit to top 1000 features
X = vectorizer.fit_transform(data['cleaned_utterance']).toarray()
y = data['intent']


In [ ]:
# Split into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"Training set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")
print(f"Testing set size: {len(X_test)}")


Training set size: 5723
Validation set size: 1226
Testing set size: 1227


# MODEL SELECTION AND TRAINING

In [ ]:
#CHECK FOR NAN VALUES
# Check for NaNs in transformed datasets
print("NaNs in X_train after imputation:", np.isnan(X_train).sum())
print("NaNs in X_val after imputation:", np.isnan(X_val).sum())
print("NaNs in X_test after imputation:", np.isnan(X_test).sum())


NaNs in X_train after imputation: 0
NaNs in X_val after imputation: 0
NaNs in X_test after imputation: 0


In [ ]:
from sklearn.impute import SimpleImputer

# Initialize the imputer to replace NaNs with 0
imputer = SimpleImputer(strategy='constant', fill_value=0)

# Fit and transform X_train
X_train = imputer.fit_transform(X_train)

# Transform X_val and X_test
X_val = imputer.transform(X_val)
X_test = imputer.transform(X_test)


In [ ]:
import pandas as pd
import numpy as np

# Check for NaN values in X_train and y_train
print(pd.isna(X_train).sum().sum())  # For features (sum of NaNs across all rows and columns)
print(pd.isna(y_train).sum())  # For target



0
1


In [ ]:
y_train = y_train.fillna(0)  # Or use another value like the mean, median, or mode


In [ ]:
print(y_train.dtypes)  # Check data type of y_train
print(y_train.unique())  # Check the unique values in y_train


object
['switch_account' 'check_refund_policy' 'track_order' 'get_refund'
 'delete_account' 'change_shipping_address' 'track_refund' 'change_order'
 'contact_customer_service' 'delivery_period' 'complaint'
 'check_cancellation_fee' 'cancel_order' 'payment_issue' 'check_invoice'
 'contact_human_agent' 'get_invoice' 'newsletter_subscription'
 'create_account' 'place_order' 'review' 'registration_problems'
 'set_up_shipping_address' 'delivery_options' 'edit_account'
 'check_payment_methods' 'recover_password' 0]


In [ ]:
y_train = y_train.astype(str)  # Convert to string type
# or
# y_train = y_train.astype(int)  # If the target variable should be integers


In [ ]:
y_val = y_val.astype(str)  # or int, depending on your choice
y_test = y_test.astype(str)  # or int, depending on your choice


In [ ]:
# Convert y_train, y_val, and y_test to consistent type (e.g., string)
y_train = y_train.astype(str)
y_val = y_val.astype(str)
y_test = y_test.astype(str)

# Now proceed with training
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Evaluate on the validation set
y_val_pred = model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)

print(f"Validation Accuracy: {val_accuracy}")


Validation Accuracy: 0.9926590538336052


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Train a Logistic Regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Evaluate on the validation set
y_val_pred = model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)

print(f"Validation Accuracy: {val_accuracy}")

# Additional evaluation metrics
print("Classification Report:\n", classification_report(y_val, y_val_pred))



Validation Accuracy: 0.9926590538336052
Classification Report:
                           precision    recall  f1-score   support

            cancel_order       1.00      0.98      0.99        59
            change_order       1.00      1.00      1.00        52
 change_shipping_address       1.00      1.00      1.00        28
  check_cancellation_fee       0.98      1.00      0.99        40
           check_invoice       0.98      1.00      0.99        47
   check_payment_methods       0.98      1.00      0.99        46
     check_refund_policy       0.98      1.00      0.99        46
               complaint       1.00      1.00      1.00        54
contact_customer_service       1.00      1.00      1.00        53
     contact_human_agent       1.00      1.00      1.00        51
          create_account       0.97      0.97      0.97        39
          delete_account       0.97      0.97      0.97        39
        delivery_options       1.00      1.00      1.00        42
         de

#MODEL EVALUATION

In [ ]:
# Test the model
y_test_pred = model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Test Accuracy: {test_accuracy}")
print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))


Test Accuracy: 0.9926650366748166

Classification Report:
                          precision    recall  f1-score   support

            cancel_order       1.00      1.00      1.00        41
            change_order       1.00      1.00      1.00        54
 change_shipping_address       1.00      0.98      0.99        54
  check_cancellation_fee       1.00      0.98      0.99        55
           check_invoice       0.98      1.00      0.99        56
   check_payment_methods       1.00      0.96      0.98        52
     check_refund_policy       1.00      1.00      1.00        39
               complaint       1.00      1.00      1.00        36
contact_customer_service       0.98      1.00      0.99        40
     contact_human_agent       1.00      1.00      1.00        35
          create_account       0.98      0.96      0.97        57
          delete_account       0.97      1.00      0.99        37
        delivery_options       0.95      1.00      0.97        36
         delivery